In [8]:
import pandas as pd
from gurobipy import *
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


families = pd.read_csv("family_data.csv")
familiesnp = np.genfromtxt('family_data.csv', delimiter=',')
familiesnp = familiesnp[1:5001]
newRow = np.full((100), 500 + (36+398)*families.iloc[0]["n_people"])
nFamilies = families['family_id'].tolist()
nDays = list(range(100,0,-1))

family_size_dict = families[['n_people']].to_dict()['n_people']
family_size_ls = list(family_size_dict.values())
cols = [f'choice_{i}' for i in range(10)]
choice_dict = families[cols].to_dict()
choice_dict_num = [{vv:i for i, vv in enumerate(di.values())} for di in choice_dict.values()]


# Computer penalities in a list
penalties_dict = {
    n: [
        0,
        50,
        50 + 9 * n,
        100 + 9 * n,
        200 + 9 * n,
        200 + 18 * n,
        300 + 18 * n,
        300 + 36 * n,
        400 + 36 * n,
        500 + 36 * n + 199 * n
    ]
    for n in range(max(family_size_dict.values())+1)
} 

penalties = np.zeros(shape=(families.shape[0], 100))
distances = pd.DataFrame(penalties, columns=nDays)
for i in families.family_id.values:
    distances.loc[i, families.iloc[i, 1:11].values] = penalties_dict[families.iloc[i,11]]
for i in families.family_id.values:
    distances.loc[i, lambda x: ~x.columns.isin(families.iloc[i, 1:11].values)] = 500 + 36 * families.iloc[i,11] + 398 * families.iloc[i,11]

favorableDays = distances.copy()
favorableDays = favorableDays <= 550
favorableDays

def getPenalty(x,y):
    return(((x-125)/400)*x**(0.5+abs(x-y)/50))

def getPenaltyFromXandDelta(x,delta):
    return(((x-125)/400)*x**(0.5+delta/50))

evaluate = {}
discretizer = 40
discretizer_range = 600
discretizer_step = discretizer_range / discretizer

for z in range(discretizer):
    #print((z+0.5)**1.05*discretizer_step)
    evaluate.update( {z : {}} )
    evaluate[z].update( {125 : 175} )            
    for j in range(125,301,1):
        for i in range(0,175):
            if(getPenaltyFromXandDelta(j,i)>(z+0.5)**1.05*discretizer_step):
                evaluate[z].update( {j : i-1} )
                break

peopleSteps = [125]+list(range(126,151,5))+list(range(175,301,25))
peopleSteps2 = [125]+peopleSteps
    
for z in range(discretizer):
    #print((z+0.5)**1.05*discretizer_step)
    evaluate.update( {z : {}} )
    for j in peopleSteps:
        if j == 125:
            evaluate[z].update( {125 : 175} )            
        for i in range(0,175):
            if(getPenaltyFromXandDelta(j+1,i)>(z+0.5)**1*discretizer_step):
                evaluate[z].update( {j : i-1} )
                break

    
                
#for z in range(discretizer):
#    x = list(evaluate[z].keys())
#   y = list(evaluate[z].values())
#    plt.plot(x,y)
    
#plt.show()    


# Create a new model
m = Model()
# Add variables
allocations = m.addVars(nFamilies,nDays)
numberOfVisitors = m.addVars(nDays,lb=125,ub=300)
deltaVisitors = m.addVars(nDays,lb=-175,ub=175)
deltaVisitorsAbs = m.addVars(nDays,lb=0,ub=175)
useHighChange = m.addVars(nDays,range(discretizer),vtype=GRB.BINARY)
numberOfPeopleBinary = m.addVars(nDays, peopleSteps, vtype=GRB.BINARY)
numberOfPeopleBinaryUpper = m.addVars(nDays, peopleSteps, vtype=GRB.BINARY)
numberOfPeopleBinaryLower = m.addVars(nDays, peopleSteps, vtype=GRB.BINARY)
dailyAllowance = m.addVars(nDays,peopleSteps,range(discretizer))
objValue = m.addVar()
m.update()

for i in nFamilies:
    m.addConstr(quicksum(allocations[i,j] for j in nDays) == 1, "must be served %s" %i)
    for j in nDays:
        m.addConstr(allocations[i,j] <= favorableDays.loc[i,j], "favorable day %s %s" %(i,j))
m.update()
           
m.addConstrs((numberOfVisitors[j] == quicksum(allocations[i,j]*familiesnp[i,11] for i in nFamilies) for j in nDays), name='nVisitors')
m.update()

m.addConstrs((deltaVisitors[j] >= numberOfVisitors[j+1] - numberOfVisitors[j] for j in nDays if j < 100), "changeInAttendance")
m.update()

m.addConstr(deltaVisitors[100]==0,"last day change")
m.update()

for j in nDays:
    m.addGenConstrAbs(deltaVisitorsAbs[j], deltaVisitors[j], "absconstr %s" %j)
m.update()

for j in nDays:
    for k in range(len(peopleSteps)):
        if k >= 1:
            m.addGenConstrIndicator(numberOfPeopleBinaryLower[j, peopleSteps[k]], True, numberOfVisitors[j], GRB.GREATER_EQUAL, peopleSteps[k-1], name = "people indicator lower %s %s" %(j,k))
        m.addGenConstrIndicator(numberOfPeopleBinaryUpper[j, peopleSteps[k]], True, numberOfVisitors[j], GRB.LESS_EQUAL, peopleSteps[k], name = "people indicator upper %s %s" %(j,k))
        m.addGenConstrAnd(numberOfPeopleBinary[j, peopleSteps[k]], [numberOfPeopleBinaryLower[j, peopleSteps[k]], numberOfPeopleBinaryUpper[j, peopleSteps[k]]])
        for l in range(discretizer):
            m.addConstr(dailyAllowance[j,peopleSteps[k],l]<=useHighChange[j,l])
            m.addConstr(dailyAllowance[j,peopleSteps[k],l]<=numberOfPeopleBinary[j,peopleSteps[k]])
m.update()

for j in nDays:
    if j<100:
        m.addConstr(deltaVisitorsAbs[j] <= quicksum(quicksum(dailyAllowance[j,peopleSteps2[k],m] * (evaluate[m][peopleSteps2[k-1]]+(numberOfVisitors[j]-peopleSteps2[k-1])*((evaluate[m][peopleSteps2[k]]-evaluate[m][peopleSteps2[k-1]])/(max(1,peopleSteps2[k]-peopleSteps2[k-1]))))  for m in range(discretizer)) for k in range(1,len(peopleSteps2))),"change limits %s" %j)
        m.addConstr(quicksum(useHighChange[j,m] for m in range(discretizer))==1, "unique change limit %s" %j)

        
m.addConstr(objValue == quicksum(quicksum(distances.loc[i,j] * allocations[i,j] for i in nFamilies) + quicksum(useHighChange[j,m] * (m+0.5)**1.0*discretizer_step for m in range(discretizer)) for j in nDays),"objective")
m.update()

m.setObjective(objValue, GRB.MINIMIZE)

m.setParam("NonConvex", 2)

m.setParam("MIPFocus", 0)
m.setParam("Heuristics", 0.9)
m.setParam("TimeLimit", 999999)
m.setParam("MIPGap",0.01)
m.setParam("Presolve", 1)
m.setParam("Method", 1)
#m.setParam("TimeLimit", 15.0);
#m.setParam("ObjBoundC", 65000.0);

start_values = pd.read_csv('sub_step_size_2.csv')
for index,row in start_values.iterrows():
    allocations[row['family_id'], row['assigned_day']].start = 1



m.optimize()

nonIntegerFamilies = set()
integerFamilies = set()

counter = 0
for i in nFamilies:
    for j in nDays:
        if(allocations[i,j].x > 0.1):
            if(allocations[i,j].x < 0.9):
                print(allocations[i,j].x)
                nonIntegerFamilies.add(i)
                break

for i in nFamilies:
    for j in nDays:
        if(allocations[i,j].x > 0.99):
            counter = counter + 1
            integerFamilies.add((i,j))

                
                
allocationsBinary = m.addVars(nonIntegerFamilies,nDays,vtype=GRB.BINARY)
for i in nonIntegerFamilies:
    m.addConstr(quicksum(allocationsBinary[i,j] - allocations[i,j] for j in nDays) == 0)

for (i,j) in integerFamilies:
    m.addConstr(allocations[i,j]==1)

m.update()

                
            
m.update()
m.optimize()
        


FileNotFoundError: [Errno 2] File b'family_data.csv' does not exist: b'family_data.csv'

In [6]:
solution = np.zeros( 5000 )
for i in nFamilies:
    for j in nDays:
        if(allocations[i,j].x > 0.5):
            solution[i] = j
            break
        
score = 0
familyScore = 0
penaltyScore = 0
for i in nFamilies:
    familyScore = familyScore + distances.loc[i, solution[i]]
for j in nDays:
    if j <100:
        penaltyScore = penaltyScore + getPenalty(numberOfVisitors[j].x, numberOfVisitors[j+1].x)
    else:
        penaltyScore = penaltyScore + getPenalty(numberOfVisitors[j].x, numberOfVisitors[j].x)
print(familyScore)
print(penaltyScore)
print(familyScore+penaltyScore)

results = list(map(int, solution))
submission = pd.read_csv("sample_submission.csv")


submission['assigned_day'] = results
family_size_dict = families[['n_people']].to_dict()['n_people']

submission = pd.read_csv("sample_submission.csv")
submission['family_size'] = submission.apply(lambda x: family_size_dict[x.family_id], axis=1)
print(submission.groupby('assigned_day').agg({'family_size': sum}).sort_values('family_size'))


submission = pd.read_csv("sample_submission.csv")
submission['assigned_day'] = results
submission.to_csv(f'submission_new.csv', index=False)

In [7]:
A

{(1, 2)}